# Extract PDFs
Logik:
* Verschiedene Templates, wie die Daten vorliegen könnten. 
* Templates werden nacheinander durchprobiert, bis 180 Stimmen exportiert wurden
* Einige PDFs sind nicht lesbar. Diese werden ausgeklammert

In [1]:
import pandas as pd
from pathlib import Path
import tabula
import json
from datetime import datetime

In [2]:
df_documents = pd.read_csv(Path('../export/abstimmungen.csv'))
df_documents['Sitzungsdatum'] = pd.to_datetime(df_documents['Sitzungsdatum'])
df_documents = df_documents.sort_values('Sitzungsdatum')

## Excludes

In [3]:
# Exclude days. They published only rastered files then.
exclude_days = [
    '2013-05-13',
    '2013-06-17',
    '2013-06-24',
    '2014-03-31',
    '2014-05-05',
    '2015-12-07',
    '2016-09-26',
    '2016-10-24',
    '2016-12-12',
    '2017-03-13',
    '2019-01-14',
    '2019-05-20',
    '2019-12-09',
    '2020-03-02',
]

exclude_list = [
  # OCR-Files (some are included in the exclude list above)
  '2475cb15a8344f13bf06af57d8df7b6a-332', 
  '825bb07a708d484e9aab0122374c96ee-332',
  '1a43a208b59c4665850ad4a13b874dc2-332',
  '7e5e5927378f436cab9d081c424005a7-332',
  'a396a35da60c4676a6e2c84fe3466e33-332',
  '6ec31a40559a450f8eba896e79ad4ce0-332',
  'f6d286de8a444d9694b1da17c5f6cbaf-332',
  'da0fb4f0ee094b6bb06b0509d4c202cb-332',
  'f6e78363414143989fadb00a4c850671-332',
  '53cdf71cb40949718f526ac334503659-332',
  '58217f34333b4ad099eba69e6689a757-332',
  '128740aaa08641cb818671086ba12407-332',
  '2ec3fefeef264dddbe5547ac7ce17861-332',
  '65b9abd5722b4304a7465253b0fe7cd1-332',
  '8fde45cf78614f1dba2c5ff1754729d5-332',
  'd8d1ac4c117845c49cd123933bac2bfc-332',
  'd4e664fd33cf4f09a51339c4a6705cf6-332',
  '60c47b4f4f1e4e939bfe245774d12044-332',
  'ca6cfe86b160424ea068599ce9a89de9-332',
  '48cd67a42b2b42ac97065313a4bc4435-332',
  'e7fcb1a2105a47eeab8f51a4706c424a-332',
  '97af4b09426f41c3940981500450dabc-332',
  '1514c9f2c9e84aeaae22f981e03b6acd-332',

  # WTF...
  'cdfb9a82b24145ed8faf017771d9c9da-332',
  '607a463667bb411495b49245cfffbafe-332',

  # Fehlerhafter Zeichensatz
  '4266a27393644bd48c311bd9692ead59-332',
  '9ba073ed8e9a4d12b993fe8b4ce961d6-332',
  '774d77917a6245a6adce75b6458532b0-332',
  '75702d2259c44ebc850b21967696ebf7-332',

  # Fehlerhaftes PDF (keine Detaildaten)
  '3dda293d605e4a62b7c4331824ebca95-332',

  # Falsch benannt
  '4398af876b1249acab10d9a50a93b6f1-332',
  '77663a0de59f4458bc670cad3f1f0beb-332',

  # Manuell exportieren
  '8b3b9b3a8055465aa6f283962a3bce1f-332',
  'd0cc72acec634165bb8f35cd81aabad8-332',
  '67d2dd24db92442ea171a4e566cc6677-332',
]

# Extract

In [4]:
header = ['platz', 'nachname', 'vorname', 'partei', 'stimme']

def extract_template(fn_in, tmpl):

    try:
        
        dfs = tabula.read_pdf_with_template(fn_in, tmpl, pandas_options={"names": header})

        # Copy Columns-Name into own Dataframe. Bug in exporter, pandas_options:names will only be used for first table
        for i in range(1, len(dfs)):
            dfx = dfs[i].copy()

            if len(dfx.columns) > 5:
                dfx = dfx.drop(columns=dfx.columns[5])

            df_header = pd.DataFrame([{
                'platz': dfx.columns[0],
                'nachname': dfx.columns[1],
                'vorname': dfx.columns[2],
                'partei': dfx.columns[3],
                'stimme': dfx.columns[4],
            }])

            # Set new Header
            dfx.columns = header

            # Concat
            dfs[i] = pd.concat([df_header, dfx])

        # Concat
        df = pd.concat(dfs).reset_index(drop=True)

        # Drop
        df = df[df.platz != 0]
        df = df[df.platz != 'Platz#']
        df = df[~df.platz.str.contains('Unnamed', na=False)]
        df = df[~df.vorname.str.contains('Vorname', na=False)]
        df = df.dropna(subset=['nachname'])

        return df, True

        
    except Exception as e:
        return None, False

In [5]:
# Templates
extraction_templates = [
    Path('./templates/extractiontemplate_2013.json'),
    Path('./templates/extractiontemplate_2013_enlarged.json'),
    Path('./templates/extractiontemplate_2013_pagebreak.json'),
    Path('./templates/extractiontemplate_2013_version3.json')
]

for i, row in df_documents[~df_documents.Sitzungsdatum.isin(exclude_days)].iterrows():

    fn_in = Path('../export/eDocuments/{}.pdf'.format(row['eDocumentID']))
    fn_out = Path('../export/eDocumentsCSV/{}.csv'.format(row['eDocumentID']))

    # Skip if already exported
    if (fn_out.exists()) or (row['eDocumentID'] in exclude_list):
        continue

    export_log = []
    success = False
    
    # Try each template. Stop, when we got 180 votes (or sometimes 170)
    for tmpl in extraction_templates:

        # Extract
        df, state = extract_template(fn_in, tmpl)

        # Did it work?
        if state == False:
            export_log.append("❌ %s" % tmpl)
            continue        

        # Check if enough votes (180 or sometimes 179)
        if (
            (len(df) == 180) or
            # Beachte Tage, an denen der Kantonsrat vorübergehend nur aus 179 Mitgliedern bestand
            ((row['Sitzungsdatum'] >= datetime(2014, 3, 3))   and (row['Sitzungsdatum'] <= datetime(2014, 5, 5))   and len(df) == 179) or
            ((row['Sitzungsdatum'] >= datetime(2015, 7, 6))   and (row['Sitzungsdatum'] <= datetime(2015, 7, 6))   and len(df) == 179) or
            ((row['Sitzungsdatum'] >= datetime(2016, 12, 19)) and (row['Sitzungsdatum'] <= datetime(2016, 12, 19)) and len(df) == 179) or
            ((row['Sitzungsdatum'] >= datetime(2018, 7, 2))   and (row['Sitzungsdatum'] <= datetime(2018, 7, 2))   and len(df) == 179) or
            ((row['Sitzungsdatum'] >= datetime(2018, 9, 10))  and (row['Sitzungsdatum'] <= datetime(2018, 9, 17))  and len(df) == 179)

        ):
            # Stire and break loop

            df.to_csv(fn_out, index=False)
            success = True

            # Reorder Templatelist. Use last template first
            extraction_templates.remove(tmpl)
            extraction_templates = [tmpl] + extraction_templates

            # Stop Template export
            break
        else:
            # Add Log. Print log, when no export did work
            export_log.append("%s (%s)" % (tmpl, len(df)))
        
    if not success:
        print("Fehlgeschlagen %s" % row['eDocumentID'])
        for l in export_log:
            print("👉 %s" % l)

print("🍩 Finito")
